
#League of Legends Winner Prediction
---

Usando dataset: https://www.kaggle.com/datasets/paololol/league-of-legends-ranked-matches

Proyecto de Inteligencia artificial creado por

*   David Camilo García


Estudiantes de la Universidad De Antioquia






### Importación de archivos de Kaggle

In [2]:
!kaggle datasets download -d paololol/league-of-legends-ranked-matches

500 - Internal Server Error


In [3]:
import zipfile
with zipfile.ZipFile("league-of-legends-ranked-matches.zip","r") as zip_ref:
     zip_ref.extractall()


## Dataset, Organización y limpieza de datos

In [4]:
#Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
plt.style.use('ggplot')
%matplotlib inline

In [5]:
#Lectura de archivos
champs = pd.read_csv("champs.csv")
matches = pd.read_csv("matches.csv")
participants = pd.read_csv('participants.csv')
stats1 = pd.read_csv('stats1.csv')
stats2 = pd.read_csv('stats2.csv')
stats = pd.concat([stats1,stats2], ignore_index=True)

#Dimensiones de las estadísticas disponibles
stats.shape

C:\Users\user\AppData\Local\Temp\ipykernel_33180\1256728992.py:6: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  stats2 = pd.read_csv('stats2.csv')


(1834517, 56)

In [6]:
stats.head()

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0
2,11,0,1055,3072,3006,3031,3046,1036,3340,7,...,3,1,2,182,14,1,0,13,5,0
3,12,0,1029,3078,3156,1001,3053,0,3340,5,...,6,6,0,106,15,0,0,3,0,0
4,13,0,3020,1058,3198,3102,1052,1026,3340,2,...,4,3,1,159,15,0,0,10,0,0


In [7]:
#Unión de datos stats, champs y matches
df = pd.merge(participants, stats, how = 'left', on = ['id'], suffixes=('', '_y'))

df = pd.merge(df , champs, how = 'left', left_on= 'championid', right_on='id'
             ,suffixes=('', '_y') )

df = pd.merge(df, matches, how = 'left', left_on = 'matchid', right_on = 'id'
              , suffixes=('', '_y'))

In [8]:
#Columnas disponibles para trabajar
df.columns

Index(['id', 'matchid', 'player', 'championid', 'ss1', 'ss2', 'role',
       'position', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood', 'name

In [9]:
#Ajuste de rol para cada partida
def final_position(col):
    if col['role'] in ('DUO_SUPPORT', 'DUO_CARRY'):
        return col['role']
    else:
        return col['position']

df['adjposition'] = df.apply(final_position, axis = 1)

In [10]:
df.head(10)

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,id_y,id_y,gameid,platformid,queueid,seasonid,duration,creation,version,adjposition
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,19,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,267,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,119,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,114,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,112,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID
5,14,10,6,72,11,4,NONE,JUNGLE,1.0,1400.0,...,72,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE
6,15,10,7,3,4,12,SOLO,TOP,1.0,3025.0,...,3,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP
7,16,10,8,103,14,4,SOLO,MID,1.0,3135.0,...,103,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID
8,17,10,9,222,7,4,DUO_CARRY,BOT,1.0,3046.0,...,222,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY
9,18,10,10,161,14,4,DUO_SUPPORT,BOT,1.0,1058.0,...,161,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT


In [11]:
df['team'] = df['player'].apply(lambda x: '1' if x <= 5 else '2')
df['team_role'] = df['team'] + ' - ' + df['adjposition']

In [12]:
df.head(10)

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,gameid,platformid,queueid,seasonid,duration,creation,version,adjposition,team,team_role
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE,1,1 - JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT,1,1 - DUO_SUPPORT
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY,1,1 - DUO_CARRY
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP,1,1 - TOP
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID,1,1 - MID
5,14,10,6,72,11,4,NONE,JUNGLE,1.0,1400.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE,2,2 - JUNGLE
6,15,10,7,3,4,12,SOLO,TOP,1.0,3025.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP,2,2 - TOP
7,16,10,8,103,14,4,SOLO,MID,1.0,3135.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID,2,2 - MID
8,17,10,9,222,7,4,DUO_CARRY,BOT,1.0,3046.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY,2,2 - DUO_CARRY
9,18,10,10,161,14,4,DUO_SUPPORT,BOT,1.0,1058.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT,2,2 - DUO_SUPPORT


In [13]:
#Removiendo roles duplicados por información perdida
remove_index = []
for i in ('1 - MID', '1 - TOP', '1 - DUO_SUPPORT', '1 - DUO_CARRY', '1 - JUNGLE',
          '2 - MID', '2 - TOP', '2 - DUO_SUPPORT', '2 - DUO_CARRY', '2 - JUNGLE'):
    df_remove = df[df['team_role'] == i].groupby('matchid').agg({'team_role':'count'})
    remove_index.extend(df_remove[df_remove['team_role'] != 1].index.values)

In [14]:
#Removiendo partidas con información mal catalogada
remove_index.extend(df[df['adjposition'] == 'BOT']['matchid'].unique())
remove_index = list(set(remove_index))

In [15]:
df = df[~df['matchid'].isin(remove_index)]
print('Partidas luego de la limpieza de datos:{}'.format(df['matchid'].nunique()))
df.shape

Partidas luego de la limpieza de datos:148638


(1486362, 76)

In [16]:
df.columns

Index(['id', 'matchid', 'player', 'championid', 'ss1', 'ss2', 'role',
       'position', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood', 'name

In [17]:
df = df[['id', 'matchid', 'player', 'name', 'adjposition', 'team_role',
         'win', 'kills', 'deaths', 'assists', 'turretkills','totdmgtochamp',
         'totheal', 'totminionskilled', 'goldspent', 'totdmgtaken', 'inhibkills',
         'pinksbought', 'wardsplaced', 'duration', 'platformid',
         'seasonid', 'version']]
df.head()

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,totminionskilled,goldspent,totdmgtaken,inhibkills,pinksbought,wardsplaced,duration,platformid,seasonid,version
0,9,10,1,Warwick,JUNGLE,1 - JUNGLE,0.0,6.0,10.0,1.0,...,42.0,10275.0,41446.0,0.0,0.0,10.0,1909,EUW1,8,7.10.187.9675
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,17.0,7975.0,17769.0,0.0,1.0,17.0,1909,EUW1,8,7.10.187.9675
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,205.0,11775.0,25627.0,0.0,1.0,13.0,1909,EUW1,8,7.10.187.9675
3,12,10,4,Fiora,TOP,1 - TOP,0.0,5.0,11.0,2.0,...,164.0,10683.0,31705.0,0.0,0.0,3.0,1909,EUW1,8,7.10.187.9675
4,13,10,5,Viktor,MID,1 - MID,0.0,2.0,8.0,2.0,...,235.0,10485.0,20585.0,0.0,0.0,10.0,1909,EUW1,8,7.10.187.9675


In [18]:
#Corrigiendo datos de wards

df_v = df.copy()

df_v['wardsplaced'] = df_v['wardsplaced'].apply(lambda x: x if x<30 else 30)
df_v['wardsplaced'] = df_v['wardsplaced'].apply(lambda x: x if x>0 else 0)

df_v['wardsplaced'].head()

0    10.0
1    17.0
2    13.0
3     3.0
4    10.0
Name: wardsplaced, dtype: float64

In [19]:
df_win_rate = df.groupby('name').agg({'win': 'sum','name': 'count',
                                     'kills':'mean','deaths':'mean',
                                     'assists':'mean'})
df_win_rate.columns = ['win' , 'total matches', 'K', 'D', 'A']
df_win_rate['win rate'] = df_win_rate['win'] / df_win_rate['total matches'] * 100
df_win_rate['KDA'] = (df_win_rate['K'] + df_win_rate['A']) / df_win_rate['D']
df_win_rate = df_win_rate.sort_values('win rate',ascending= False)
df_win_rate = df_win_rate[['total matches', 'win rate' , 'K' , 'D', 'A', 'KDA']]

In [20]:
print('Top 10 win rate')
print(df_win_rate.head(10))
print('Least 10 win rate')
print(df_win_rate.tail(10))

Top 10 win rate
          total matches   win rate         K         D          A       KDA
name                                                                       
Ivern              6671  55.868685  2.639934  4.156798  13.299505  3.834547
Anivia             6433  53.971708  6.372610  4.769625   7.544847  2.917935
Xerath             5108  53.621770  7.314996  5.444401   8.553837  2.914707
Ahri              30841  53.545605  7.264591  5.551621   7.711706  2.697644
Sona              11847  53.135815  2.971638  5.735629  13.947750  2.949875
Skarner            1720  53.081395  4.790698  5.021512   9.222674  2.790668
Zilean             4797  52.991453  3.155931  4.727747  11.909944  3.186693
KogMaw             6440  52.717391  7.931988  6.589596   7.125155  2.284987
Janna             21182  52.714569  0.872580  4.018365  14.476867  3.819824
Pantheon           8872  52.569883  8.068756  6.466186   6.837241  2.305222
Least 10 win rate
         total matches   win rate         K         D 

In [21]:
df_2 = df.sort_values(['matchid','adjposition'], ascending = [1,1])

In [22]:
df_2

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,totminionskilled,goldspent,totdmgtaken,inhibkills,pinksbought,wardsplaced,duration,platformid,seasonid,version
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,205.0,11775.0,25627.0,0.0,1.0,13.0,1909,EUW1,8,7.10.187.9675
8,17,10,9,Jinx,DUO_CARRY,2 - DUO_CARRY,1.0,15.0,3.0,9.0,...,191.0,14375.0,17655.0,2.0,1.0,6.0,1909,EUW1,8,7.10.187.9675
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,17.0,7975.0,17769.0,0.0,1.0,17.0,1909,EUW1,8,7.10.187.9675
9,18,10,10,VelKoz,DUO_SUPPORT,2 - DUO_SUPPORT,1.0,4.0,5.0,19.0,...,72.0,11675.0,13443.0,0.0,5.0,25.0,1909,EUW1,8,7.10.187.9675
0,9,10,1,Warwick,JUNGLE,1 - JUNGLE,0.0,6.0,10.0,1.0,...,42.0,10275.0,41446.0,0.0,0.0,10.0,1909,EUW1,8,7.10.187.9675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834518,1865603,187588,9,Riven,JUNGLE,2 - JUNGLE,0.0,2.0,11.0,3.0,...,53.0,9430.0,31790.0,0.0,0.0,8.0,2534,NA1,5,5.20.0.291
1834510,1865595,187588,1,Twisted Fate,MID,1 - MID,1.0,3.0,14.0,7.0,...,190.0,11900.0,29670.0,0.0,0.0,10.0,2534,NA1,5,5.20.0.291
1834516,1865601,187588,7,Annie,MID,2 - MID,0.0,11.0,9.0,7.0,...,189.0,12575.0,21003.0,0.0,0.0,13.0,2534,NA1,5,5.20.0.291
1834513,1865598,187588,4,Malphite,TOP,1 - TOP,1.0,7.0,4.0,22.0,...,150.0,12880.0,26215.0,0.0,3.0,11.0,2534,NA1,5,5.20.0.291


In [23]:
df_2['shift 1'] = df_2['name'].shift()
df_2['shift -1'] = df_2['name'].shift(-1)

In [24]:
df_2

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,totdmgtaken,inhibkills,pinksbought,wardsplaced,duration,platformid,seasonid,version,shift 1,shift -1
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,25627.0,0.0,1.0,13.0,1909,EUW1,8,7.10.187.9675,None,Jinx
8,17,10,9,Jinx,DUO_CARRY,2 - DUO_CARRY,1.0,15.0,3.0,9.0,...,17655.0,2.0,1.0,6.0,1909,EUW1,8,7.10.187.9675,Draven,Nami
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,17769.0,0.0,1.0,17.0,1909,EUW1,8,7.10.187.9675,Jinx,VelKoz
9,18,10,10,VelKoz,DUO_SUPPORT,2 - DUO_SUPPORT,1.0,4.0,5.0,19.0,...,13443.0,0.0,5.0,25.0,1909,EUW1,8,7.10.187.9675,Nami,Warwick
0,9,10,1,Warwick,JUNGLE,1 - JUNGLE,0.0,6.0,10.0,1.0,...,41446.0,0.0,0.0,10.0,1909,EUW1,8,7.10.187.9675,VelKoz,Skarner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834518,1865603,187588,9,Riven,JUNGLE,2 - JUNGLE,0.0,2.0,11.0,3.0,...,31790.0,0.0,0.0,8.0,2534,NA1,5,5.20.0.291,Xin Zhao,Twisted Fate
1834510,1865595,187588,1,Twisted Fate,MID,1 - MID,1.0,3.0,14.0,7.0,...,29670.0,0.0,0.0,10.0,2534,NA1,5,5.20.0.291,Riven,Annie
1834516,1865601,187588,7,Annie,MID,2 - MID,0.0,11.0,9.0,7.0,...,21003.0,0.0,0.0,13.0,2534,NA1,5,5.20.0.291,Twisted Fate,Malphite
1834513,1865598,187588,4,Malphite,TOP,1 - TOP,1.0,7.0,4.0,22.0,...,26215.0,0.0,3.0,11.0,2534,NA1,5,5.20.0.291,Annie,Olaf


In [25]:
#Definimos los Enfrentamientos singulares
def matchup(x):
    if x['player'] <= 5:
        if x['name'] < x['shift -1']:
            name_return = x['name'] + ' vs ' + x['shift -1']
        else:
            name_return = x['shift -1'] + ' vs ' + x['name']
    else:
        if x['name'] < x['shift 1']:
            name_return = x['name'] + ' vs ' + x['shift 1']
        else:
            name_return = x['shift 1'] + ' vs ' + x['name']
    return name_return

df_2['matchup'] = df_2.apply(matchup, axis = 1)

df_2['win_adj'] = df_2.apply(lambda x: x['win'] if x['name'] == x['matchup'].split(' vs')[0]
                            else "2", axis = 1)


In [26]:
drop = df_2.index[df_2['win_adj'] == "2"].tolist()

In [27]:
df_2.head(10)

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,pinksbought,wardsplaced,duration,platformid,seasonid,version,shift 1,shift -1,matchup,win_adj
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,1.0,13.0,1909,EUW1,8,7.10.187.9675,None,Jinx,Draven vs Jinx,0.0
8,17,10,9,Jinx,DUO_CARRY,2 - DUO_CARRY,1.0,15.0,3.0,9.0,...,1.0,6.0,1909,EUW1,8,7.10.187.9675,Draven,Nami,Draven vs Jinx,2
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,1.0,17.0,1909,EUW1,8,7.10.187.9675,Jinx,VelKoz,Nami vs VelKoz,0.0
9,18,10,10,VelKoz,DUO_SUPPORT,2 - DUO_SUPPORT,1.0,4.0,5.0,19.0,...,5.0,25.0,1909,EUW1,8,7.10.187.9675,Nami,Warwick,Nami vs VelKoz,2
0,9,10,1,Warwick,JUNGLE,1 - JUNGLE,0.0,6.0,10.0,1.0,...,0.0,10.0,1909,EUW1,8,7.10.187.9675,VelKoz,Skarner,Skarner vs Warwick,2
5,14,10,6,Skarner,JUNGLE,2 - JUNGLE,1.0,3.0,3.0,9.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Warwick,Viktor,Skarner vs Warwick,1.0
4,13,10,5,Viktor,MID,1 - MID,0.0,2.0,8.0,2.0,...,0.0,10.0,1909,EUW1,8,7.10.187.9675,Skarner,Ahri,Ahri vs Viktor,2
7,16,10,8,Ahri,MID,2 - MID,1.0,13.0,4.0,8.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Viktor,Fiora,Ahri vs Viktor,1.0
3,12,10,4,Fiora,TOP,1 - TOP,0.0,5.0,11.0,2.0,...,0.0,3.0,1909,EUW1,8,7.10.187.9675,Ahri,Galio,Fiora vs Galio,0.0
6,15,10,7,Galio,TOP,2 - TOP,1.0,4.0,5.0,11.0,...,1.0,7.0,1909,EUW1,8,7.10.187.9675,Fiora,Cassiopeia,Fiora vs Galio,2


In [28]:
df_2 = df_2.drop(drop)
df_2.head(10)

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,pinksbought,wardsplaced,duration,platformid,seasonid,version,shift 1,shift -1,matchup,win_adj
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,1.0,13.0,1909,EUW1,8,7.10.187.9675,None,Jinx,Draven vs Jinx,0.0
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,1.0,17.0,1909,EUW1,8,7.10.187.9675,Jinx,VelKoz,Nami vs VelKoz,0.0
5,14,10,6,Skarner,JUNGLE,2 - JUNGLE,1.0,3.0,3.0,9.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Warwick,Viktor,Skarner vs Warwick,1.0
7,16,10,8,Ahri,MID,2 - MID,1.0,13.0,4.0,8.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Viktor,Fiora,Ahri vs Viktor,1.0
3,12,10,4,Fiora,TOP,1 - TOP,0.0,5.0,11.0,2.0,...,0.0,3.0,1909,EUW1,8,7.10.187.9675,Ahri,Galio,Fiora vs Galio,0.0
15,24,11,6,Caitlyn,DUO_CARRY,2 - DUO_CARRY,1.0,12.0,5.0,4.0,...,1.0,6.0,1693,EUW1,8,7.10.187.9675,Cassiopeia,Ziggs,Caitlyn vs Cassiopeia,1.0
18,27,11,9,Karma,DUO_SUPPORT,2 - DUO_SUPPORT,1.0,3.0,7.0,13.0,...,3.0,12.0,1693,EUW1,8,7.10.187.9675,Ziggs,Evelynn,Karma vs Ziggs,1.0
14,23,11,5,Evelynn,JUNGLE,1 - JUNGLE,0.0,19.0,3.0,8.0,...,0.0,12.0,1693,EUW1,8,7.10.187.9675,Karma,Warwick,Evelynn vs Warwick,0.0
17,26,11,8,Syndra,MID,2 - MID,1.0,5.0,7.0,4.0,...,2.0,11.0,1693,EUW1,8,7.10.187.9675,Yasuo,Twisted Fate,Syndra vs Yasuo,1.0
13,22,11,4,Twisted Fate,TOP,1 - TOP,0.0,9.0,7.0,11.0,...,0.0,9.0,1693,EUW1,8,7.10.187.9675,Syndra,Wukong,Twisted Fate vs Wukong,0.0


In [29]:
df_matchup = df_2.groupby(['adjposition', 'matchup']).agg({'win_adj': 'sum', 'matchup': 'count'})
df_matchup.columns = ['win matches', 'total matches']
df_matchup['total matches'] = df_matchup['total matches']
df_matchup['win rate'] = df_matchup['win matches'] /  df_matchup['total matches']  * 100
df_matchup['dominant score'] = df_matchup['win rate']
df_matchup['dominant score (ND)'] = abs(df_matchup['dominant score'])
df_matchup = df_matchup[df_matchup['total matches'] > df_matchup['total matches'].sum()*0.0001]

df_matchup = df_matchup.sort_values('dominant score (ND)', ascending = False)
df_matchup = df_matchup[['total matches', 'dominant score']]
df_matchup = df_matchup.reset_index()

for i in df_matchup['adjposition'].unique():
        print('\n{}:'.format(i))
        print(df_matchup[df_matchup['adjposition'] == i].iloc[:,1:].head(5))


MID:
             matchup  total matches dominant score
0      Annie vs Ryze             80           75.0
1       Ahri vs Ryze            319      71.159875
8       Ekko vs Ryze            130      63.846154
11      Fizz vs Ryze            158      63.291139
12  Kassadin vs Ryze            147      63.265306

TOP:
                matchup  total matches dominant score
2   Gangplank vs Singed            102      65.686275
5       Yasuo vs Yorick            109      65.137615
9      Irelia vs Kennen            186       63.44086
10         Jax vs Poppy            109      63.302752
16    Camille vs Gragas             77      62.337662

JUNGLE:
              matchup  total matches dominant score
3   Evelynn vs Rengar            118      65.254237
4   Hecarim vs Twitch             89      65.168539
7          Nunu vs Vi             95      64.210526
15  Elise vs Volibear             88           62.5
18    Ivern vs KhaZix            367      62.125341

DUO_SUPPORT:
                 matchu

In [30]:
df_matchup['adjposition'].unique()

df_matchup_TOP = df_matchup.loc[df_matchup['adjposition'] == 'TOP']
df_matchup_JUNGLE = df_matchup.loc[df_matchup['adjposition'] == 'JUNGLE']
df_matchup_MID = df_matchup.loc[df_matchup['adjposition'] == 'MID']
df_matchup_DUO_CARRY = df_matchup.loc[df_matchup['adjposition'] == 'DUO_CARRY']
df_matchup_DUO_SUPPORT = df_matchup.loc[df_matchup['adjposition'] == 'DUO_SUPPORT']

print(df_matchup_TOP.shape)
print(df_matchup_JUNGLE.shape)
print(df_matchup_MID.shape)
print(df_matchup_DUO_CARRY.shape)
print(df_matchup_DUO_SUPPORT.shape)

(525, 4)
(391, 4)
(435, 4)
(138, 4)
(272, 4)


In [31]:
df_matchup

,adjposition,matchup,total matches,dominant score
0,MID,Annie vs Ryze,80,75.0
1,MID,Ahri vs Ryze,319,71.159875
2,TOP,Gangplank vs Singed,102,65.686275
3,JUNGLE,Evelynn vs Rengar,118,65.254237
4,JUNGLE,Hecarim vs Twitch,89,65.168539
...,...,...,...,...
1756,MID,Ryze vs Twisted Fate,78,35.897436
1757,JUNGLE,Graves vs Ivern,410,35.609756
1758,TOP,Shen vs Teemo,189,35.449735
1759,MID,Azir vs Lux,150,35.333333


In [32]:
df_matchup['matchup'] = df_matchup['matchup'].astype("string")

df_matchup.dtypes

adjposition               object
matchup           string[python]
total matches              int64
dominant score            object
dtype: object

In [88]:
df_2

,id,matchid,player,name,adjposition,team_role,win,kills,deaths,assists,...,pinksbought,wardsplaced,duration,platformid,seasonid,version,shift 1,shift -1,matchup,win_adj
2,11,10,3,Draven,DUO_CARRY,1 - DUO_CARRY,0.0,7.0,8.0,5.0,...,1.0,13.0,1909,EUW1,8,7.10.187.9675,None,Jinx,Draven vs Jinx,0.0
1,10,10,2,Nami,DUO_SUPPORT,1 - DUO_SUPPORT,0.0,0.0,2.0,12.0,...,1.0,17.0,1909,EUW1,8,7.10.187.9675,Jinx,VelKoz,Nami vs VelKoz,0.0
5,14,10,6,Skarner,JUNGLE,2 - JUNGLE,1.0,3.0,3.0,9.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Warwick,Viktor,Skarner vs Warwick,1.0
7,16,10,8,Ahri,MID,2 - MID,1.0,13.0,4.0,8.0,...,0.0,8.0,1909,EUW1,8,7.10.187.9675,Viktor,Fiora,Ahri vs Viktor,1.0
3,12,10,4,Fiora,TOP,1 - TOP,0.0,5.0,11.0,2.0,...,0.0,3.0,1909,EUW1,8,7.10.187.9675,Ahri,Galio,Fiora vs Galio,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834515,1865600,187588,6,Jinx,DUO_CARRY,2 - DUO_CARRY,0.0,11.0,10.0,2.0,...,3.0,13.0,2534,NA1,5,5.20.0.291,Tristana,Morgana,Jinx vs Tristana,0.0
1834517,1865602,187588,8,Blitzcrank,DUO_SUPPORT,2 - DUO_SUPPORT,0.0,4.0,8.0,10.0,...,0.0,5.0,2534,NA1,5,5.20.0.291,Morgana,Xin Zhao,Blitzcrank vs Morgana,0.0
1834518,1865603,187588,9,Riven,JUNGLE,2 - JUNGLE,0.0,2.0,11.0,3.0,...,0.0,8.0,2534,NA1,5,5.20.0.291,Xin Zhao,Twisted Fate,Riven vs Xin Zhao,0.0
1834516,1865601,187588,7,Annie,MID,2 - MID,0.0,11.0,9.0,7.0,...,0.0,13.0,2534,NA1,5,5.20.0.291,Twisted Fate,Malphite,Annie vs Twisted Fate,0.0


In [95]:
x = df_2["matchup"]
y = df_2["win_adj"]
from sklearn.model_selection import train_test_split

df_matchup.dropna(inplace=True)

x_train, x_test, y_train, y_test = train_test_split(x,y)
x_train = x_train.reset_index()

In [96]:
x_train

,index,matchup
0,328566,Graves vs Skarner
1,1771203,Draven vs Ezreal
2,181864,Fizz vs Kassadin
3,567472,Sejuani vs Yasuo
4,1654146,Camille vs Master Yi
...,...,...
557381,644304,Fiora vs Kayle
557382,915685,Diana vs Lee Sin
557383,692019,Alistar vs Rakan
557384,474087,Darius vs Riven


In [105]:
def predict(x_train,x_test):
  x_test = x_test.copy()
  x_test = x_test.values.tolist()
  y_predicted = []
  for rows in x_test:
    matchup = x_train[(x_train["matchup"] == rows)]["dominant score"].values
    if len(matchup) == 0:
      y_predicted.append(np.random.randint(0,1))
    elif len(matchup) == 1 and matchup[0] < 50:
      y_predicted.append(0)
    else:
      y_predicted.append(1)
  return y_predicted


y_pred = predict(df_matchup,x_test)


In [106]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pickle

#y_test = y_test.values.tolist()

print(accuracy_score(y_test,y_pred))



0.5257594350793343


## 2024-1 Modelos

In [ ]:
# Agregar la columna 'dominant score' al dataframe original mediante un merge al revés
df_original = pd.merge(df_2, df_matchup[['matchup', 'dominant score']], on='matchup', how='left')

# Seleccionar las columnas especificadas
df_original = df_original[['matchid', 'player', 'matchup', 'win_adj', 'dominant score']]

# Eliminar las filas con valores NaN en la columna "dominant score"
df_original = df_original.dropna(subset=['dominant score'])
df_original = df_original.dropna(subset=['win_adj'])

# Convertir la columna "win_adj" a entero
df_original['win_adj'] = df_original['win_adj'].astype(int)

# Convertir la columna "dominant score" a flotante
df_original['dominant score'] = df_original['dominant score'].astype(float)

# Restablecer los índices después de eliminar filas
df_original = df_original.reset_index(drop=True)


# Guardar train_data en un archivo CSV
train_data = df_original[['matchup', 'dominant score', 'win_adj']].copy()  # Seleccionar las columnas relevantes


In [153]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(df_original[['matchup', 'dominant score']], df_original['win_adj'], test_size=0.3, random_state=1)

# Crear y entrenar el modelo RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train[['dominant score']], y_train)  # Entrenar solo con la columna "dominant score"

def predict(model, x_test, threshold=0.5):
    # Obtener las probabilidades estimadas de cada clase para cada instancia de prueba
    proba = model.predict_proba(x_test[['dominant score']])
    
    # Predicción basada en el umbral
    y_pred = np.where(proba[:, 1] > threshold, 1, 0)
    
    return y_pred

# Utilizar la función predict para hacer predicciones en el conjunto de prueba
y_pred = predict(model, x_test, threshold=0.5)


In [159]:
# Guardar datos de prueba en archivos CSV
import pickle

train_data.to_csv("train_data.csv", index=False)
x_test.to_csv("test_data_input.csv", index=False)
y_test.to_csv("test_data_target.csv", index=False, header=['win_adj'])  # Especificar el encabezado para la columna "win_adj"

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)


In [158]:
print(accuracy_score(y_test,y_pred))

0.5167930206465549
